In [ ]:
import numpy as np
import os
import glob
from librosa.filters import mel as librosa_mel_fn

import joblib
import glob
datasplitARCTIC = list(np.load("../../seq2seq-vc/notebooks/data_split_ARCTIC.npy", allow_pickle=True))
datasplitVCTK = list(np.load("../../seq2seq-vc/notebooks/data_split_VCTK.npy", allow_pickle=True))
datasplit = []
for i in range(3):
    datasplit += [datasplitARCTIC[i]+datasplitVCTK[i]]

In [ ]:
base_dir = "/mntcephfs/"
audio_dirs = {
    "LibriTTS": "/mntcephfs/lee_dataset/tts/LibriTTS_R/",
    "L2-ARCTIC": "/mntcephfs/lab_data/shoinoue/Dataset/L2-ARCTIC/",
    "CMU-ARCTIC": "/mntcephfs/lab_data/shoinoue/Dataset/CMU-ARCTIC/",
    "SPAccenT_ARCTIC": "/mntcephfs/lab_data/shoinoue/Dataset/PD-AST/SLT/",
    "SPAccenT_VCTK": "/mntcephfs/lab_data/shoinoue/Dataset/PD-AST/SLT_add/",
}
mode_list = ["train", "dev", "test"]

In [ ]:
save = True
target_dir = "../LibriTTS_ARCTIC_SPAccenT/"

for mode in mode_list:
    print(mode)
    arcticfiles = datasplit[mode_list.index(mode)]
    fulltext = "" 
    for key in audio_dirs:
        audio_dir = audio_dirs[key]
        # if key!="LibriTTS" and mode in ["dev", "test"]:
        #     continue
        if key=="LibriTTS":
            spks = [os.path.basename(a) for a in glob.glob(audio_dir + f"{mode}*/*")]
            files = []
            for spk in spks:
                files += glob.glob(audio_dir + f"{mode}*/{spk}/*/*.wav")
        elif key in ["L2-ARCTIC", "CMU-ARCTIC", "SPAccenT_ARCTIC", "SPAccenT_VCTK"]:
            spks = [os.path.basename(a) for a in glob.glob(audio_dir + f"*")]
            try:
                spks.remove("suitcase_corpus")
            except ValueError:
                pass
            files = []
            for spk in spks:
                files += glob.glob(audio_dir + f"{spk}/wav/*.wav")
            new_files = []
            for path in files:
                bn = os.path.basename(path)[:-4]
                if bn in arcticfiles:
                    new_files += [path]
            files = new_files
        files.sort()
        files = files[::10] if mode in ["dev", "test"] else files
        print(f"{key}: {len(files)}")
        for fn in files:
            name = fn[len(base_dir):-4]
            fulltext += f"{name}|unchi\n"
    if save:
        if mode=="train":
            modename = "training"
        elif mode=="dev":
            modename = "validation"
        elif mode=="test":
            modename = "test"
        f = open(f"{target_dir}{modename}.txt", "w")
        f.write(fulltext)
        f.close()
    print()